<a href="https://colab.research.google.com/github/SashaBreygina/Project1_PDS/blob/main/Project1_Deliverable1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install requests beautifulsoup4 transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [13]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util

In [14]:
# Load Sentence Transformer Model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
def extract_text_from_url(url):
    """Extracts text from a given URL using BeautifulSoup."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        page_text = " ".join([p.text for p in soup.find_all("p")])
        return page_text[:1000]  # Limit text for analysis
    except Exception as e:
        return f"Failed to fetch content: {str(e)}"

In [16]:
def check_fact_with_google(query):
    """Calls Google's Fact Check API to verify claims."""
    API_KEY = "YOUR_GOOGLE_FACT_CHECK_API_KEY"
    URL = f"https://factchecktools.googleapis.com/v1alpha1/claims:search?query={query}&key={API_KEY}"

    try:
        response = requests.get(URL)
        data = response.json()
        if "claims" in data:
            return 1.0 if any(claim["claimReview"][0]["textualRating"] == "True" for claim in data["claims"]) else 0.5
        return 0.2  # Low score if no verification found
    except:
        return 0.3  # Default score if API fails

In [17]:
def evaluate_credibility(url, user_query):
    """Evaluates credibility of an article by scoring multiple factors."""
    # Extract content from URL
    page_text = extract_text_from_url(url)
    if "Failed to fetch" in page_text:
        return {"score": 0, "explanation": page_text}

    # Step 1: Domain Authority Check (Placeholder)
    domain_trust = 60  # Replace with actual Moz API score if available

    # Step 2: Content Relevance (Semantic Similarity)
    similarity_score = util.pytorch_cos_sim(model.encode(user_query), model.encode(page_text)).item()

    # Step 3: Fact-Checking (Google Fact Check API)
    fact_check_score = check_fact_with_google(page_text)

    # Final Credibility Score (Weighted)
    credibility_score = (0.4 * (domain_trust / 100)) + (0.3 * similarity_score) + (0.3 * fact_check_score)

    explanation = (
        f"Domain Trust: {domain_trust}/100, "
        f"Relevance Score: {similarity_score:.2f}, "
        f"Fact Check Score: {fact_check_score:.2f}"
    )

    return {"score": round(credibility_score, 2), "explanation": explanation}


In [18]:
# Example Usage
url = "https://www.bbc.com/news/science-environment-56901261"
user_query = "Climate change effects"
print(evaluate_credibility(url, user_query))

{'score': 0.45, 'explanation': 'Domain Trust: 60/100, Relevance Score: 0.51, Fact Check Score: 0.20'}
